## Adversarial Attacks on ResNet50

This notebook demonstrates several adversarial attacks (FGSM, PGD, CW, DeepFool) against a pre-trained ResNet50 model on a subset of ImageNet data.

In [ ]:
import torch
import torch.utils.data as data
import torchvision.models as models
import torchvision.datasets as datasets

from attacks.fgsm import FGSM
from attacks.pgd import PGD
from attacks.cw import CW
from attacks.deepfool import DeepFool

from utils.helpers import preprocess, set_seed, get_fixed_parameter, load_imagenet_classes
from utils.evaluation import evaluate_attack, evaluate_all_attacks, print_accuracy_confidence
from utils.visualization import (
    plot_accuracy_vs_param, visualize_generic_sweep,
    visualize_pgd_heatmaps, visualize_comparison_results, 
    visualize_deepfool_heatmaps
)

## Configuration

In [ ]:
IMAGENET_VAL_DIR = './imagenet_val/'
LABEL_MAP_FILE = './imagenet_classes.txt'
BATCH_SIZE = 4     # How many images to process at once
NUM_IMAGES = 4    # Max number of images to test
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 806

# Set random seed for reproducibility
set_seed(SEED, DEVICE)

# Check device
print(f"Using device: {DEVICE}")

## Load model and data

### Load Pre-trained model (ResNet50)

In [ ]:
# Load a pre-trained ResNet50 model
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model = model.to(DEVICE)
model.eval()

print("ResNet50 model loaded successfully.")

### Load ImageNet subset

In [ ]:
# Create a dataset object
imagenet_data = datasets.ImageFolder(IMAGENET_VAL_DIR, preprocess)
print(f"Found {len(imagenet_data)} images in {IMAGENET_VAL_DIR}")
print(f"Class mapping (first 5): {list(imagenet_data.class_to_idx.items())[:5]}")

# Use a subset of the data
subset_indices = list(range(min(NUM_IMAGES, len(imagenet_data))))
imagenet_subset = data.Subset(imagenet_data, subset_indices)

# Create a DataLoader
test_loader = data.DataLoader(imagenet_subset, batch_size=BATCH_SIZE, shuffle=False)
print(f"Created DataLoader with {len(imagenet_subset)} images.")

### Load class labels

In [ ]:
imagenet_classes = load_imagenet_classes(LABEL_MAP_FILE)
print(f"Loaded {len(imagenet_classes)} class names from index file.")
print("Sample class names:", list(imagenet_classes.items())[:5])

## Parameter impact analysis

In [ ]:
subset_indices_param_test = list(range(min(NUM_IMAGES, len(imagenet_data))))
imagenet_subset_param_test = data.Subset(imagenet_data, subset_indices_param_test)
param_test_loader = data.DataLoader(imagenet_subset_param_test, batch_size=BATCH_SIZE, shuffle=False)
print(f"\nParameter testing will use {NUM_IMAGES} images.")

### FGSM Parameter Impact (Epsilon)

In [ ]:
# Controls the magnitude of the perturbation. Larger eps = stronger attack but more visible noise.
FGSM_EPSILONS = [0.001, 0.01, 0.07, 0.1, 0.3, 0.5]

In [ ]:
fgsm_results = {}
clean_data_for_fgsm = None

print("\n--- Testing FGSM Epsilons ---")
for eps in FGSM_EPSILONS:
    print(f"\nTesting FGSM with epsilon = {eps}")
    fgsm_attack = FGSM(model, eps=eps)
    results, clean_results = evaluate_attack(fgsm_attack, param_test_loader, model, DEVICE, NUM_IMAGES)
    fgsm_results[eps] = results
    if clean_data_for_fgsm is None:
        clean_data_for_fgsm = clean_results
print("----------------------------")

In [ ]:
# Prepare data for plotting
fgsm_accuracies = [res['accuracy'] for res in fgsm_results.values()]

# Plot FGSM Accuracy vs Epsilon
plot_accuracy_vs_param(FGSM_EPSILONS, fgsm_accuracies, param_label='Epsilon', title='FGSM Accuracy vs. Epsilon')

# Visualize FGSM examples for different epsilons
num_samples_to_show = min(3, NUM_IMAGES)
visualize_generic_sweep(
    num_samples_to_show=num_samples_to_show, 
    param_values=FGSM_EPSILONS, 
    attack_results=fgsm_results, 
    clean_data=clean_data_for_fgsm, 
    imagenet_classes=imagenet_classes, 
    attack_name="FGSM",
    param_label_in_subplot_title="eps",
    param_display_name_in_suptitle="Epsilon"
)

### PGD Parameter Impact (Epsilon, Alpha, Steps)

In [ ]:
# Maximum perturbation allowed. Larger eps = stronger attack but more visible noise.
PGD_EPSILONS = [0.001, 0.01, 0.07, 0.1]
# Step size for each iteration. Smaller alpha = more iterations needed to reach eps.
PGD_ALPHAS = [0.001, 0.01, 0.1]
# Number of iterations. More steps = potentially stronger attack but slower.
PGD_STEPS = [10, 30]

Run attacks

In [ ]:
pgd_results = {}
clean_data_for_pgd = None

print("\n--- Testing PGD Parameter Combinations ---")
for eps in PGD_EPSILONS:
    for alpha in PGD_ALPHAS:
        for steps in PGD_STEPS:
            params = (eps, alpha, steps)
            print(f"\nTesting PGD with epsilon = {eps}, alpha = {alpha}, steps = {steps}")
            pgd_attack = PGD(model, eps=eps, alpha=alpha, steps=steps)
            results, clean_results = evaluate_attack(pgd_attack, param_test_loader, model, DEVICE, NUM_IMAGES)
            pgd_results[params] = results
            if clean_data_for_pgd is None:
                clean_data_for_pgd = clean_results

print("-----------------------------------------")

Plot results

In [ ]:
# Take median of alpha and steps for plotting
fixed_alpha = get_fixed_parameter(PGD_ALPHAS)
fixed_steps = get_fixed_parameter(PGD_STEPS)

pgd_accuracies_for_epsilon_plot = []
pgd_results_for_epsilon_visualization = {}

for eps_val in PGD_EPSILONS:
    # Construct the parameter tuple for the current epsilon and fixed alpha/steps
    current_params_key = (eps_val, fixed_alpha, fixed_steps)
    pgd_accuracies_for_epsilon_plot.append(pgd_results[current_params_key]['accuracy'])
    pgd_results_for_epsilon_visualization[eps_val] = pgd_results[current_params_key]

# Plot PGD Accuracy vs Epsilon (for the fixed alpha and steps)
plot_accuracy_vs_param(
    PGD_EPSILONS,
    pgd_accuracies_for_epsilon_plot,
    param_label='Epsilon',
    title=f'PGD Accuracy vs. Epsilon (alpha={fixed_alpha}, steps={fixed_steps})'
)

# Visualize PGD examples for different epsilons (for the fixed alpha and steps)
num_samples_to_show = min(3, NUM_IMAGES)
visualize_generic_sweep(
    num_samples_to_show=num_samples_to_show,
    param_values=PGD_EPSILONS,
    attack_results=pgd_results_for_epsilon_visualization,
    clean_data=clean_data_for_pgd,
    imagenet_classes=imagenet_classes,
    attack_name=f"PGD (α={fixed_alpha}, steps={fixed_steps})",
    param_label_in_subplot_title="eps",
    param_display_name_in_suptitle="Epsilon"
)

Heatmap Visualization

In [ ]:
visualize_pgd_heatmaps(pgd_results, PGD_EPSILONS, PGD_ALPHAS, PGD_STEPS)

### CW Parameter Impact (C)

In [ ]:
# CW:
# Controls the trade-off between perturbation magnitude and classification confidence. Higher c = stronger attack.
CW_CS = [100, 1000, 10000]
# Number of optimization steps.
CW_STEPS = 100
# Learning rate for the optimizer.
CW_LR = 0.01
# Confidence parameter. Increase to make misclassification more confident. 0 means just misclassification.
CW_KAPPA = 0

In [ ]:
cw_results = {}
clean_data_for_cw = None

print("\n--- Testing CW C Values ---")
print(f"(Using reduced steps={CW_STEPS} for faster testing)")
for c_val in CW_CS:
    print(f"Testing CW with c = {c_val}, kappa = {CW_KAPPA}, steps = {CW_STEPS}, lr = {CW_LR}")
    cw_attack = CW(model, c=c_val, kappa=CW_KAPPA, steps=CW_STEPS, lr=CW_LR)
    results, clean_results = evaluate_attack(cw_attack, param_test_loader, model, DEVICE, NUM_IMAGES)
    cw_results[c_val] = results
    if clean_data_for_cw is None:
        clean_data_for_cw = clean_results
print("-------------------------")

In [ ]:
# Prepare data for plotting
cw_accuracies = [res['accuracy'] for res in cw_results.values()]

# Plot CW Accuracy vs C
plot_accuracy_vs_param(
    CW_CS,
    cw_accuracies,
    param_label='C value',
    title=f'CW Accuracy vs. C (kappa={CW_KAPPA}, steps={CW_STEPS}, lr={CW_LR})'
)

# Visualize CW examples for different C values
num_samples_to_show = min(3, NUM_IMAGES)
visualize_generic_sweep(
    num_samples_to_show=num_samples_to_show,
    param_values=CW_CS,
    attack_results=cw_results,
    clean_data=clean_data_for_cw,
    imagenet_classes=imagenet_classes,
    attack_name=f"CW",
    param_label_in_subplot_title="c",
    param_display_name_in_suptitle="C",
    suptitle_extra_info=f"(kappa={CW_KAPPA}, steps={CW_STEPS}, lr={CW_LR})"
)

### DeepFool parameter impact (Steps, Overshoot)

In [ ]:
# DeepFool:
# Maximum number of iterations to find the boundary. Beware: increasing will make process painfully slow.
DF_STEPS = [1, 2, 5]
# Factor to push the image slightly over the boundary.
DF_OVERSHOOT = [0.01, 0.1, 0.5]
# Model specific; ResNet50 is trained on ImageNet with 1000 classes.
NUM_CLASSES = 1000

In [ ]:
deepfool_results = {}
clean_data_for_deepfool = None

print("\n--- Testing DeepFool Parameter Combinations ---")
for steps_val in DF_STEPS:
    for overshoot_val in DF_OVERSHOOT:
        params = (steps_val, overshoot_val)
        print(f"\nTesting DeepFool with steps = {steps_val}, overshoot = {overshoot_val}")
        deepfool_attack = DeepFool(model, steps_val, overshoot_val, NUM_CLASSES)
        results, clean_results = evaluate_attack(deepfool_attack, param_test_loader, model, DEVICE, NUM_IMAGES)
        deepfool_results[params] = results
        if clean_data_for_deepfool is None:
            clean_data_for_deepfool = clean_results
print("-----------------------------------------")

In [ ]:
# Prepare data for plotting (median overshoot for steps plot, median steps for overshoot plot)
fixed_steps_df = get_fixed_parameter(DF_STEPS)
num_samples_to_show_df_overshoot = min(3, NUM_IMAGES)
df_accuracies_for_overshoot_plot = []
df_results_for_overshoot_visualization = {}

for overshoot_val in DF_OVERSHOOT:
    current_params_key = (fixed_steps_df, overshoot_val)
    df_accuracies_for_overshoot_plot.append(deepfool_results[current_params_key]['accuracy'])
    df_results_for_overshoot_visualization[overshoot_val] = deepfool_results[current_params_key]

# Plot DeepFool Accuracy vs. Overshoot (for fixed median steps)
plot_accuracy_vs_param(
    DF_OVERSHOOT,
    df_accuracies_for_overshoot_plot,
    param_label='Overshoot',
    title=f'DeepFool Accuracy vs. Overshoot (Steps={fixed_steps_df})'
)

# Visualize DeepFool examples for different overshoot (for fixed median steps)
visualize_generic_sweep(
    num_samples_to_show=num_samples_to_show_df_overshoot,
    param_values=DF_OVERSHOOT,
    attack_results=df_results_for_overshoot_visualization,
    clean_data=clean_data_for_deepfool,
    imagenet_classes=imagenet_classes,
    attack_name=f"DeepFool (Steps={fixed_steps_df})",
    param_label_in_subplot_title="Overshoot",
    param_display_name_in_suptitle="Overshoot"
)

In [ ]:
visualize_deepfool_heatmaps(deepfool_results, DF_STEPS, DF_OVERSHOOT)

## Attacks comparison

Fixed parameters for comparison

In [ ]:
# FGSM
FGSM_EPSILON = 0.1

# PGD
PGD_EPSILON = 0.1
PGD_ALPHA = 0.01
PGD_STEPS = 10

# CW
CW_C = 1000
CW_STEPS = 100
CW_LR = 0.01
CW_KAPPA = 0

# DeepFool
DF_STEPS = 2
DF_OVERSHOOT = 0.1
NUM_CLASSES = 1000

In [ ]:
# Initialize attacks dictionary
attacks = {}

# Add attacks using the fixed parameters
attacks['FGSM'] = FGSM(model, FGSM_EPSILON)
attacks['PGD'] = PGD(model, PGD_EPSILON, PGD_ALPHA, PGD_STEPS)
attacks['CW'] = CW(model, CW_C, CW_KAPPA, CW_STEPS, CW_LR)
attacks['DeepFool'] = DeepFool(model, DF_STEPS, DF_OVERSHOOT, NUM_CLASSES)

In [ ]:
results = evaluate_all_attacks(
    attacks,
    test_loader,
    model,
    DEVICE,
    NUM_IMAGES,
    imagenet_classes
)

### Analyze Results

In [ ]:
# Visualize sample results
num_samples_to_show = min(5, results['clean']['total'] if results['clean']['total'] > 0 else 0)
if num_samples_to_show > 0:
    visualize_comparison_results(num_samples_to_show, results, attacks, imagenet_classes)

In [ ]:
print_accuracy_confidence(results, attacks)